In [ ]:
# Remember to install tianshou first
!pip install tianshou==0.4.8
!pip install gym

# Overview
In reinforcement learning, the agent interacts with environments to improve itself. In this tutorial we will concentrate on the environment part. Although there are many kinds of environments or their libraries in DRL research, Tianshou chooses to keep a consistent API with [OPENAI Gym](https://gym.openai.com/).

<div align=center>
<img src="https://tianshou.readthedocs.io/en/master/_images/rl-loop.jpg", title="The agents interacting with the environment">

<a> The agents interacting with the environment </a>
</div>

In Gym, an environment receives an action and returns next observation and reward. This process is slow and sometimes can be the throughput bottleneck in a DRL experiment.


Tianshou provides vectorized environment wrapper for a Gym environment. This wrapper allows you to make use of multiple cpu cores in your server to accelerate the data sampling.

In [ ]:
from tianshou.env import SubprocVectorEnv
import numpy as np
import gym
import time

num_cpus = [1,2,5]
for num_cpu in num_cpus:
  env = SubprocVectorEnv([lambda: gym.make('CartPole-v0') for _ in range(num_cpu)])
  env.reset()
  sampled_steps = 0
  time_start = time.time()
  while sampled_steps < 1000:
    act = np.random.choice(2, size=num_cpu)
    obs, rew, done, info = env.step(act)
    if np.sum(done):
      env.reset(np.where(done)[0])
    sampled_steps += num_cpu
  time_used = time.time() - time_start
  print("{}s used to sample 1000 steps if using {} cpus.".format(time_used, num_cpu))

You may notice that the speed doesn't increase linearly when we add subprocess numbers. There are multiple reasons behind this. One reason is that synchronize exection causes straggler effect. One way to solve this would be to use asynchronous mode. We leave this for further reading if you feel interested.

Note that SubprocVectorEnv should only be used when the environment exection is slow. In practice, DummyVectorEnv (or raw Gym environment) is actually more efficient for a simple environment like CartPole because now you avoid both straggler effect and the overhead of communication between subprocesses.

# Usages
## Initialisation
Just pass in a list of functions which return the initialised environment upon called.

In [ ]:
from tianshou.env import DummyVectorEnv
# In Gym
env = gym.make("CartPole-v0")

# In Tianshou
def helper_function():
  env = gym.make("CartPole-v0")
  # other operations such as env.seed(np.random.choice(10))
  return env

envs = DummyVectorEnv([helper_function for _ in range(5)])
print(envs)

## EnvPool supporting
Besides integrated environment wrappers, Tianshou also fully supports [EnvPool](https://github.com/sail-sg/envpool/). Explore its Github page yourself.

## Environment exection and resetting
The only difference between Vectorized environments and standard Gym environments is that passed in actions and returned rewards/observations are also vectorized.

In [ ]:
# In Gym, env.reset() returns a single observation.
print("In Gym, env.reset() returns a single observation.")
print(env.reset())

# In Tianshou, envs.reset() returns stacked observations.
print("========================================")
print("In Tianshou, envs.reset() returns stacked observations.")
print(envs.reset())

obs, rew, done, info = envs.step(np.random.choice(2, size=num_cpu))
print(info)

If we only want to execute several environments. The `id` argument can be used.

In [ ]:
print(envs.step(np.random.choice(2, size=3), id=[0,3,1]))

# Further Reading
## Other environment wrappers in Tianshou


*   ShmemVectorEnv: use share memory instead of pipe based on SubprocVectorEnv;
*   RayVectorEnv: use Ray for concurrent activities and is currently the only choice for parallel simulation in a cluster with multiple machines.

Check the [documentation](https://tianshou.readthedocs.io/en/master/api/tianshou.env.html) for details.

## Difference between synchronous and asynchronous mode (How to choose?)
Explanation can be found at the [Parallel Sampling](https://tianshou.readthedocs.io/en/master/tutorials/cheatsheet.html#parallel-sampling) tutorial.